# 🚀 Indexación de Productos con Vectores Densos y Dispersos

En este notebook vamos a indexar el dataset completo de productos de motos usando el sistema de retriever con vectores densos (OpenAI) y dispersos (SPLADE) para búsquedas híbridas avanzadas.

## Características de este dataset:
- **ID**: Entero (`id_producto`) - Compatible directamente con Qdrant
- **Vectores**: Densos + Dispersos para búsqueda híbrida
- **Campos**: Marca, modelo, categoría, precios, dimensiones
- **Texto**: Campo `texto` para generar embeddings
- **Upsert**: Habilitado para actualizaciones automáticas

## 1. Configuración y Imports

In [ ]:
# --- Configuración Inicial ---
import pandas as pd
import logging

# Configurar logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Cargar datos (ajusta la ruta según tu estructura)
print("📁 Cargando dataset de productos...")
df_productos = pd.read_pickle("../data/curated/shopify_data_to_index.pkl")

# Verificar estructura del dataset
print(f"📊 Dataset cargado:")
print(f"   📈 Total productos: {len(df_productos):,}")
print(f"   🔑 Columnas: {list(df_productos.columns)}")

# Mostrar ejemplo de registro
ejemplo_producto = df_productos.iloc[0].to_dict()
print(f"\n📋 Ejemplo de producto:")
print(f"   🆔 ID: {ejemplo_producto.get('id_producto')}")
print(f"   📄 Título: {ejemplo_producto.get('titulo', 'N/A')[:60]}...")
print(f"   🏷️ Marca: {ejemplo_producto.get('marca', 'N/A')}")
print(f"   📂 Categoría: {ejemplo_producto.get('categoria', 'N/A')}")
print(f"   💰 Precio: ${ejemplo_producto.get('precio', 0):,.0f}")
print(f"   🛞 Es llanta: {ejemplo_producto.get('es_llanta', False)}")

In [ ]:
# ## Seleccion de muestra

# categorias = [
#     'ELECTRICO / ENCENDIDO',
#     'SUJECION / SELLOS / UNIVERSALES',
#     'CARROCERIA / PLASTICOS',
#     'MOTOR INTERNO',
#     'CONTROLES / MANDOS',
#     'FRENOS'
#     'SUSPENSION / DIRECCION'
#     'ILUMINACION / SEÑALIZACION'
#     'EMBRAGUE / CLUTCH'
#     'TRANSMISION SECUNDARIA'
#     'OTROS'
# ]

# marcas = [
#     "YAMAHA",   
#     "BAJAJ",
#     "KYMCO",
#     "HERO",
#     "VICTORY",
#     "TVS",
#     "KAWASAKI",
#     "MICHELIN",
#     "PIRELLI",
#     "BENELLI",
#     "KONTROL",
#     "METZELER",
#     "GENERICO"
# ]


# df_productos = df_productos[
#     df_productos['marca_original'].isin(marcas) & df_productos['categoria'].isin(categorias)
# ].reset_index(drop=True)
# df_productos['id_producto'] = df_productos['id_producto'].astype(int)
# df_productos.shape

In [ ]:
# import asyncio
# import logging
# from src.scripts import ProductIndexer

# # Configurar logging
# logging.basicConfig(level=logging.INFO)
# logger = logging.getLogger(__name__)

# indexer = ProductIndexer(
#     collection_name="productos_ejemplo",
#     batch_size=50,
#     overwrite_collection=True  # Sobrescribir si existe
# )

# res = await indexer.index_products(
#         df=df_productos.head(10), 
#         text_field="texto",
#         id_field="id_producto"
#     )

## Indexar todo el catalo de repuestos

In [ ]:
df_productos = pd.read_pickle("../data/curated/shopify_data_to_index.pkl")

df_productos = df_productos.dropna(subset=["id_producto", "texto"])
df_productos = df_productos[df_productos["texto"].apply(lambda x: str(x).strip() != "")]
df_productos.shape

In [ ]:
import asyncio
import logging
from src.scripts import ProductIndexer

# Configurar logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

indexer = ProductIndexer(
    batch_size=100,
    overwrite_collection=True  # Sobrescribir si existe
)

res = await indexer.index_products(
        df=df_productos, 
        text_field="texto",
        id_field="id_producto"
    )

### Catalogos 

In [ ]:
import json, re, ast
from pathlib import Path
import pandas as pd
import numpy as np

# ---------------------- helpers ----------------------
def _isna(x):
    return x is None or (isinstance(x, float) and np.isnan(x))

def as_list(x):
    """Convierte una celda en lista.
    - list -> list
    - NaN/None -> []
    - str con representaciones de lista -> literal_eval si aplica, si no, intenta split por '|', ','.
    - otro -> [str(x)]
    """
    if _isna(x):
        return []
    if isinstance(x, list):
        return [str(v).strip() for v in x if str(v).strip()]
    if isinstance(x, str):
        s = x.strip()
        # intenta parsear como lista literal
        if (s.startswith("[") and s.endswith("]")) or (s.startswith("(") and s.endswith(")")):
            try:
                parsed = ast.literal_eval(s)
                if isinstance(parsed, (list, tuple)):
                    return [str(v).strip() for v in parsed if str(v).strip()]
            except Exception:
                pass
        # si viene separado por | o ,
        if "|" in s:
            return [t.strip() for t in s.split("|") if t.strip()]
        if "," in s:
            return [t.strip() for t in s.split(",") if t.strip()]
        return [s] if s else []
    # cualquier otro
    return [str(x).strip()]

def parse_dim_dict(x):
    """Devuelve dict con posibles keys 'ancho','alto','rin' desde celda dimensiones o dimensiones_str."""
    if _isna(x):
        return {}
    if isinstance(x, dict):
        return {k:int(v) for k,v in x.items() if v not in (None,"") and str(v).isdigit()}
    if isinstance(x, str):
        s = x.strip()
        # intenta json/literal
        if (s.startswith("{") and s.endswith("}")):
            try:
                d = json.loads(s)
                if isinstance(d, dict):
                    return {k:int(v) for k,v in d.items() if str(v).isdigit()}
            except Exception:
                try:
                    d = ast.literal_eval(s)
                    if isinstance(d, dict):
                        return {k:int(v) for k,v in d.items() if str(v).isdigit()}
                except Exception:
                    pass
        # intenta extraer "130/70-17" o "130 70 17" o "130/70 R17"
        m = re.search(r'(?P<ancho>\d{2,3})\s*[\/ ]\s*(?P<alto>\d{2,3})\s*[-Rr xX]?\s*(?P<rin>\d{2})', s)
        if m:
            return {k:int(v) for k,v in m.groupdict().items()}
        # "rin 17"
        r = re.search(r'\brin\s*(?P<rin>\d{2})\b', s, flags=re.IGNORECASE)
        if r:
            return {"rin": int(r.group("rin"))}
    return {}

def safe_int(x):
    try:
        return int(x)
    except Exception:
        return None

def unique_dim_dicts(dim_tuples):
    """Dedup de combinaciones (ancho, alto, rin) a lista de dicts sin None."""
    seen = set()
    out = []
    for a, b, r in dim_tuples:
        key = (a, b, r)
        if key in seen:
            continue
        seen.add(key)
        d = {}
        if a is not None: d["ancho"] = a
        if b is not None: d["alto"] = b
        if r is not None: d["rin"]  = r
        if d:  # solo guarda si tiene al menos 1 campo
            out.append(d)
    # ordena por rin, luego ancho, luego alto (cuando existan)
    def _sort_key(d):
        return (d.get("rin", 0), d.get("ancho", 0), d.get("alto", 0))
    out.sort(key=_sort_key)
    return out

In [ ]:
df_productos = pd.read_pickle("../data/curated/shopify_data_to_index.pkl")

# ---------------------- extracción desde df_productos ----------------------
# Columnas esperadas (ajusta si tus nombres difieren mínimamente)
col_marcas_list = "marcas_lista"
col_modelos_list= "modelos_lista"
col_categoria   = "categoria"
col_subcat      = "subcategoria"
col_tipo        = "tipo_repuesto"
col_dim         = "dimensiones"
col_dim_str     = "dimensiones_str"

if not isinstance(df_productos, pd.DataFrame):
    raise RuntimeError("df_productos no está definido como DataFrame.")

#### 1. Categorias

In [ ]:
# 1) categorias
categorias = list(set(df_productos[col_categoria]))
categorias

#### 2 Subcategorias por categoria

In [ ]:
# 2) Subcategorias por categoria
subcategorias_por_categoria = {}
for categoria in categorias:
    subcategorias_por_categoria[categoria] = list(df_productos[df_productos[col_categoria] == categoria][col_subcat].unique())
subcategorias_por_categoria

#### 3 Marcas

In [ ]:
## marcas
marcas = sorted({
    item.strip()
    for lista in df_productos[col_marcas_list].dropna()
    for item in lista if isinstance(lista, list)
})

marcas

In [ ]:
marcas_llantas = ["MICHELIN", "PIRELLI", "KONTROL", "METZELER"]

#### Modeos por marca

In [ ]:
from glob import glob

files = glob("../data/results/curated/consolidacion_referencias/*.json")

modelos_por_marca = {}
for file in files:
    with open(file, "r") as f:
        data = json.load(f)
        modelos_por_marca[data["marca"]] = [d["referencia"] for d in data["grupos"]]

len(modelos_por_marca)

In [ ]:
len(marcas) - len(marcas_llantas)

#### Tipo de Repuestos

In [ ]:
tipos_repuesto = sorted(df_productos[col_tipo].unique())
tipos_repuesto

#### Dimensiones de llantas

In [ ]:
dim_tuples = set()
if col_dim in df_productos.columns or col_dim_str in df_productos.columns:
    # filtra solo llantas si hay flag o por categoría
    if col_es_llanta in df_productos.columns:
        df_ll = df_productos[df_productos[col_es_llanta] == True]
    else:
        df_ll = df_productos[df_productos[col_categoria].astype(str).str.upper() == "LLANTA"]

    for _, row in df_ll.iterrows():
        d = {}
        if col_dim in row and not _isna(row[col_dim]):
            d = parse_dim_dict(row[col_dim])
        if not d and col_dim_str in row and not _isna(row[col_dim_str]):
            d = parse_dim_dict(row[col_dim_str])

        a = safe_int(d.get("ancho"))
        b = safe_int(d.get("alto"))
        r = safe_int(d.get("rin"))
        if any(v is not None for v in (a, b, r)):
            dim_tuples.add((a, b, r))

dimensiones_llantas_ = unique_dim_dicts(dim_tuples)
dimensiones_llantas = {"ancho": set(), "alto": set(), "rin": set()}
for d in dimensiones_llantas_:
    if d.get("ancho"):
        dimensiones_llantas["ancho"].add(d.get("ancho"))
    if d.get("alto"):
        dimensiones_llantas["alto"].add(d.get("alto"))
    if d.get("rin"):
        dimensiones_llantas["rin"].add(d.get("rin"))

for d in dimensiones_llantas:
    dimensiones_llantas[d] = sorted(dimensiones_llantas[d])

dimensiones_llantas

### Consolidando catalogos

In [ ]:
catalogos_dict = {
    "marcas": marcas,
    "marcas_llantas": marcas_llantas,
    "categorias": categorias,
    "modelos_por_marca": modelos_por_marca,
    "subcategorias_por_categoria": subcategorias_por_categoria,
    "tipos_repuesto": tipos_repuesto,
    "dimensiones_llantas": dimensiones_llantas,
}

In [ ]:
catalogos_dict

In [ ]:
ruta = Path("../data/curated/catalogos.json")
ruta.parent.mkdir(parents=True, exist_ok=True)  # crea directorios si no existen
with ruta.open("w", encoding="utf-8") as f:
    json.dump(catalogos_dict, f, ensure_ascii=False, indent=2)

In [ ]:
CATEGORY_DESCRIPTIONS = {
    "ELECTRICO / ENCENDIDO": """
    Sistema eléctrico y de encendido de la motocicleta. Incluye componentes para 
    generar, almacenar y distribuir energía eléctrica: baterías, bobinas, CDI, 
    reguladores, bujías, motor de arranque, relé de arranque, volante magnético, 
    escobillas, sensores (TPS, velocidad, oxígeno, temperatura, acelerador, inclinación), 
    socket, capuchón de bujía, plato de bobinas, switch de neutro y presión de aceite.
    """.strip(),
    
    "FRENOS": """
    Sistema de frenado completo de la motocicleta. Componentes para detener 
    o reducir la velocidad: pastillas de freno, bandas de freno, discos de freno, 
    bomba de freno, leva de freno, varilla de freno, portabandas, pedal de frenos, 
    eje abrebandas, varilla porta bandas, mordaza de freno.
    """.strip(),
    
    "MOTOR INTERNO": """
    Componentes internos del motor de combustión. Piezas que están dentro 
    del bloque del motor: cilindro, pistón, anillos, biela, cigüeñal, árbol de levas, 
    válvulas, balancines, distribución, cadenilla, tensor cadenilla, eje crank, 
    piñón cigüeñal, piñón de distribución, piñón contrabalanceador, eje balanceador, 
    bomba de aceite, varilla de empuje, mirilla de aceite.
    """.strip(),
    
    "TRANSMISION SECUNDARIA": """
    Sistema de transmisión de potencia desde la caja de cambios a la rueda trasera. 
    Componentes: kit de arrastre, sprockets, cadena de transmisión, guardacadena, 
    tensor de cadena, porta sprocket, caucho porta sprocket, deslizador de cadena, 
    piñón de transmisión, piñón de la corona, correa de transmisión.
    """.strip(),
    
    "EMBRAGUE / CLUTCH": """
    Sistema de embrague para transmisión de potencia del motor. Componentes: 
    clutch, disco clutch, separador clutch, campana clutch, clutch primario/trasero, 
    clutch automático, clutch de arranque, disco separador (no de freno).
    """.strip(),
    
    "TRANSMISION PRIMARIA / CAJA": """
    Sistema de transmisión primaria y caja de cambios. Componentes: piñón primario, 
    piñón de caja de cambios, selector de cambios, piñón impulsor de arranque, 
    piñón de neutro, eje primario, eje de cambios, eje selector cambios, 
    leva de cambios, rollers, correa/polea, leva polea, bendix, piñón motor de arranque.
    """.strip(),
    
    "ALIMENTACION": """
    Sistema de alimentación de combustible y carburación/inyección. Componentes: 
    carburador, kit carburador, aguja de aire/cortina/flotador, flotador, boquerel, 
    válvula de aire, filtro de gasolina, llave de gasolina, filtro de gases, 
    tanque de gasolina, bomba gasolina, cuerpo de inyección, sensor de gasolina.
    """.strip(),
    
    "ILUMINACION / SEÑALIZACION": """
    Sistema de iluminación y señalización de la motocicleta. Componentes: 
    farola, bombillos, direccionales, stop, tapa stop, vidrio velocímetro, 
    aro farola, claxon, pito, luces delanteras y traseras.
    """.strip(),
    
    "SUSPENSION / DIRECCION": """
    Sistema de suspensión y dirección de la motocicleta. Componentes para 
    absorber impactos y dirigir: amortiguadores, barras de suspensión, botellas de suspensión, 
    retén telescopio, cunas de dirección, kit cunas de dirección, eje tijera, 
    tijera, varilla torque, resortes de moto, guardapolvo.
    """.strip(),
    
    "CARROCERIA / PLASTICOS": """
    Elementos externos y estéticos de la motocicleta. Componentes de la 
    carrocería: guardabarros, carenajes, tapas, cola guardabarro, visor, aletas, 
    aleta tanque, carcasa, guardapiernas, canastilla, posapiés, caucho de posapiés, 
    caucho sillín, agarradera, defensa, protector de moto.
    """.strip(),
    
    "CONTROLES / MANDOS": """
    Sistemas de control y operación de la motocicleta. Componentes para 
    manejar y controlar: manubrio, maniguetas, comandos, cables (acelerador, freno, clutch, 
    choke, carburador, bomba aceite, RPM, sillín, YPVS, descompresor), pedal de cambios, 
    pedal crank, levas (clutch, pedal, choke, descompresor, selector de cambios), 
    caja de acelerador, carretel de acelerador, botón interruptor, switch encendido.
    """.strip(),
    
    "FLUIDOS / FILTRACION / LUBRICACION": """
    Fluidos, filtración y lubricación del motor y componentes. Incluye: 
    aceites (motor, suspensión, transmisión), grasa, lubricante de cadena, 
    filtros (aire, aceite, centrífugo, gases, rotatorio), desengrasante de cadena, 
    mirilla de aceite, bomba aceite.
    """.strip(),
    
    "SUJECION / SELLOS / UNIVERSALES": """
    Elementos de fijación, sellado y conexión. Componentes auxiliares: 
    tornillos, tuercas, arandelas, o-rings, retenes, empaques, espárragos, 
    pasadores, balineras, antivibrante, tapón de moto, rodamientos, abrazaderas, 
    pin para moto, bujes, disco ajuste.
    """.strip(),
    
    "INSTRUMENTACION": """
    Instrumentos de medición y transmisión de velocidad. Componentes: 
    velocímetro, tacómetro, caja piñón velocímetro, cubierta velocímetro, 
    cables de velocímetro y tacómetro.
    """.strip(),
    
    "ESCAPE": """
    Sistema de escape de gases del motor. Componentes: 
    mofle, protector de mofle, empaque mofle, silenciador, 
    tubos de escape y accesorios relacionados.
    """.strip(),
    
    "ESPEJOS": """
    Espejos retrovisores para visibilidad y seguridad. Incluye: 
    espejos retrovisores izquierdo y derecho, soportes de espejos, 
    cristales de repuesto y accesorios de montaje.
    """.strip(),
    
    "SOPORTES / BRACKETS": """
    Soportes y brackets de montaje para diversos componentes. 
    Elementos estructurales de soporte y fijación para diferentes 
    partes de la motocicleta.
    """.strip(),
    
    "REFRIGERACION": """
    Sistema de refrigeración del motor. Componentes: 
    bomba de agua, eje bomba de agua, piñón bomba de agua, 
    termostato, ventilador, radiador y mangueras de refrigeración.
    """.strip(),
    
    "SOPORTE / ESTACIONAMIENTO": """
    Sistemas de soporte y estacionamiento de la motocicleta. 
    Componentes: gato central, gato lateral, soportes de estacionamiento 
    y accesorios relacionados.
    """.strip(),
    
    "RINES / ACCESORIOS LLANTA": """
    Rines y accesorios relacionados con las llantas. Componentes: 
    rin de moto, válvula de llanta, eje llanta, sets de radios, 
    accesorios de montaje y balanceado.
    """.strip(),
    
    "ACCESORIOS / ELECTRONICA AUXILIAR": """
    Accesorios y electrónica auxiliar no esencial. Componentes: 
    radio para moto, cable de seguridad, accesorios de personalización 
    y electrónicos adicionales.
    """.strip(),
    
    "LLANTA": """
    Neumáticos para motocicletas. Llantas clasificadas por tipo de uso: 
    scooter, pisteras (pista), doble propósito off-road, enduro. 
    Se clasifican por medidas (ancho/perfil/diámetro), tipo de construcción 
    (radial/diagonal) y aplicación específica.
    """.strip(),
    
    "OTROS": """
    Categoría para productos que no encajan en las clasificaciones anteriores. 
    Repuestos diversos, componentes especializados o productos sin 
    categorización específica definida.
    """.strip()
}

In [ ]:
len(CATEGORY_DESCRIPTIONS)

In [ ]:
set(CATEGORY_DESCRIPTIONS) - set(categorias), set(categorias) - set(CATEGORY_DESCRIPTIONS)